# try question answer system

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from pinecone import Pinecone

In [2]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [3]:
embeddings = download_hugging_face_embeddings()

In [4]:
PINECONE_API_KEY = "bb77b29f-7fe6-4390-9510-cec577216f50"
pc = Pinecone(api_key=PINECONE_API_KEY)  

index_name="medical-chatbot"
index = pc.Index(index_name)

In [9]:
llm=CTransformers(model="\\Users\\Ranjan Jena\\Desktop\\Work\\GENAI\\Projects\\Medical-Chatbot-Using-LLM\\model\\llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [21]:
def retriver_results(query, top_res):
    # create the query vector
    xq = embeddings.embed_query(query)
    print(f"embeddings {len(xq)}")
    # now query
    xc = index.query(vector=xq, top_k=top_res, include_metadata=True, namespace="real", include_values=True)

    contexts = []
    contexts = contexts + [
            x['metadata']['text'] for x in xc['matches']
        ]
    
    querydocs = []
    from langchain.docstore.document import Document

    for eachcont in contexts:
        doc =  Document(page_content=eachcont, metadata={"source": "local"})
        querydocs.append(doc)

    return querydocs, "".join(contexts)

In [11]:
from langchain.chains.question_answering import load_qa_chain
chain=load_qa_chain(llm,chain_type="stuff")

# pass result chunk after similarity search to question answer chain to get output
def retrieve_answers(query):
    querydocs, context = retriver_results(query,3)
    response=chain.run(input_documents=querydocs,question=query)
    return response

In [22]:
our_query = "What are Acne?"
retrieve_answers(our_query)

embeddings 384


'Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.'

# Try 2 with prompt engineering

In [12]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [13]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# PROMPT.format(context=context,question=question)
# chain_type_kwargs={"prompt": PROMPT}

In [14]:
from langchain.chains.question_answering import load_qa_chain
chain=load_qa_chain(llm,chain_type="stuff")

# pass result chunk after similarity search to question answer chain to get output
def retrieve_answers(query):
    querydocs,context = retriver_results(query,2)
    input_prompts = PROMPT.format(context=context,question=query)
    # response=chain.run(input_documents=querydocs,question=input_prompts)
    response=llm(input_prompts)
    return response

In [15]:
our_query = "How to resolve pipmples?"
retrieve_answers(our_query)

"The best way to resolve pimples is by practicing good hygiene, using gentle skin care products, and avoiding picking or squeezing them. It's also important to see a dermatologist if the pimples are severe or persistent."